# Transformer for Summarization (Part II)


In Part I of the "Transformer for Summarization" assignment, we implemented a custom Transformer encoder-decoder model and trained it on a subset of the XSum dataset for news summarization. We then evaluated the model's validation accuracy but did not perform any generation with it. In this follow-up assignment, we will:

1. Learn how to efficiently enable the Transformer encoder-decoder model to generate sequences.
2. Fine-tune another Transformer encoder-decoder model based on the pretrained language model T5 for the same task.
3. Compare the performance of the fine-tuned model against our previous custom model that was trained from scratch.

**Note:** Ensure you run this notebook on a CUDA GPU to optimize training speed. For instance, you can use a GPU instance on Google Colab.


## Preparations

Before we begin, follow the steps below to set up the necessary packages, import libraries, configure Google Drive (if needed), load the dataset, and process it. We have also provided the reference implementation of the scaled dot-product attention and multi-head attention, which you were asked to implement in Part I.

In [ ]:
#@title Install Packages

!python -m pip install datasets==2.11.0 transformers==4.16.2 tokenizers==0.13.2 evaluate==0.4.0 rouge_score==0.1.2 einops==0.6.0

%load_ext autoreload
%autoreload 2

In [ ]:
#@title Import Packages

import os
import base64
import functools
import gzip
import json
import time

import numpy as np
import evaluate

import einops
import torch
from datasets import load_dataset
from torch import nn
from torch.nn import functional as F
from transformers import AutoTokenizer, DataCollatorForSeq2Seq, Seq2SeqTrainer, Seq2SeqTrainingArguments, T5ForConditionalGeneration
from transformers.modeling_utils import PreTrainedModel
from transformers.configuration_utils import PretrainedConfig
from transformers.modeling_outputs import Seq2SeqLMOutput

TO_SAVE = {"time": time.time()}

In [ ]:
#@title Test Utilities

def vector_all_close(x, y, rtol=1e-3):
    return ((x - y).norm(p=2, dim=-1) <= rtol * (x.norm(p=2, dim=-1) + y.norm(p=2, dim=-1))).all().item()

In [ ]:
#@title Mount Google Drive

from google.colab import drive, files
drive.mount('/content/drive/')

Please save the model we trained for Part I (https://drive.google.com/file/d/1LKkbuAN2fI5wWKywcw0vDHhvCeKYuuBI/view?usp=share_link) into `/content/drive/MyDrive/cs182_hw_summarization_part2/`

In [ ]:
assert os.path.isfile("/content/drive/MyDrive/cs182_hw_summarization_part2/hw11.pytorch_model.bin")

In [ ]:
#@title Load Data

xsum = load_dataset("xsum")
blob = b'...'
train_indices, validation_indices, test_indices = json.loads(gzip.decompress(base64.b64decode(blob)).decode("utf-8"))
dataset_train = xsum["train"].select(train_indices)
dataset_validation = xsum["validation"].select(validation_indices)
dataset_test = xsum["test"].select(test_indices)


In [ ]:
#@title Process Data

tokenizer = AutoTokenizer.from_pretrained("t5-small")

def tokenize(tokenizer, examples):
    inputs = [doc for doc in examples["document"]]
    model_inputs = tokenizer(inputs, max_length=1024, truncation=True)
    labels = tokenizer(examples["summary"], max_length=128, truncation=True)
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenize_fn = functools.partial(tokenize, tokenizer)

dataset_train = dataset_train.map(tokenize_fn, batched=True)
dataset_validation = dataset_validation.map(tokenize_fn, batched=True)
dataset_test = dataset_test.map(tokenize_fn, batched=True)

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model="t5-small")

In [ ]:
#@title Scaled Dot-Product Attention

future_mask = torch.triu(torch.zeros([1024, 1024]).fill_(float("-inf")), 1)

def scaled_dot_product_attention(q, k, v, key_padding_mask=None, causal=False):
    d_head = q.size(-1)
    s = einops.einsum(q, k, "n tl dh, n sl dh -> n tl sl") / d_head ** 0.5
    if key_padding_mask is not None:
        s = s.masked_fill(
            key_padding_mask.unsqueeze(1).to(torch.bool),
            float("-inf"),
        )
    if causal:
        attn_mask = future_mask[: s.size(1), : s.size(2)].to(s)
        s += attn_mask.unsqueeze(0)
    a = F.softmax(s, dim=-1, dtype=torch.float32).type_as(s)
    return einops.einsum(a, v, "n tl sl, n sl dh -> n tl dh")

In [ ]:
#@title Multi-head Attention

class MultiheadAttention(nn.Module):
    def __init__(self, d_model, n_heads):
        super().__init__()
        self.q_proj = nn.Linear(d_model, d_model)
        self.k_proj = nn.Linear(d_model, d_model)
        self.v_proj = nn.Linear(d_model, d_model)
        self.o_proj = nn.Linear(d_model, d_model)
        self.n_heads = n_heads
        self.d_head = d_model // n_heads
    
    def forward(self, q, k, v, key_padding_mask=None, causal=False):
        q = self.q_proj(q)
        k = self.k_proj(k)
        v = self.v_proj(v)
        q = einops.rearrange(q, "b tl (nh dh) -> (b nh) tl dh", nh=self.n_heads)
        k = einops.rearrange(k, "b sl (nh dh) -> (b nh) sl dh", nh=self.n_heads)
        v = einops.rearrange(v, "b sl (nh dh) -> (b nh) sl dh", nh=self.n_heads)
        if key_padding_mask is not None:
            key_padding_mask = einops.repeat(
                key_padding_mask, "b sl -> (b nh) sl", nh=self.n_heads)
        o = scaled_dot_product_attention(q, k, v, key_padding_mask, causal)
        o = einops.rearrange(o, "(b nh) tl dh -> b tl (nh dh)", nh=self.n_heads)
        return self.o_proj(o)

## Efficient Decoding in Transformers

Recall that the Transformer encoder-decoder model we trained is a conditional language model. Given the source sequence $\mathbf{x}$ and the target tokens before the $i$-th position, the Transformer models the probability distribution of the $i$-th token:

$$\Pr(y_i|\mathbf{x}, y_1,\dots, y_{i-1})$$

A naive approach to sequence generation would be to use this conditional language model as a black box in any generation algorithm (e.g., random sampling, greedy decoding, or beam search). Each time we need to compute $\Pr(y_i|\mathbf{x}, y_1,\dots, y_{i-1})$, we would perform a complete encoder forward pass with $\mathbf{x}$ as input, and a decoder forward pass with $y_1,\dots,y_{i-1}$ as input. However, this method would result in a significant computational cost for generating each sequence. Due to the quadratic time complexity of attention with respect to sequence length, the time complexity of this approach in random sampling would be $\Theta(n_\text{tgt}(n_\text{src}^2 + n_\text{src}n_\text{tgt}+ n_\text{tgt}^2))$.

To reduce redundant computations, let's leverage the properties of Transformers. First, observe that for a given input sequence $\mathbf{x}$, the encoder output $\mathrm{enc}(\mathbf{x})$ remains constant. As a result, we don't need to recompute it during every iteration. By computing it only once, we can reduce the computational cost to $\Theta(n_\text{src}^2 + n_\text{tgt}(n_\text{src}n_\text{tgt}+ n_\text{tgt}^2))$.

The decoder side is more complex, as each iteration appends an additional token to the decoder input. However, we will demonstrate an interesting property: when a new token $y_i$ is appended to the decoder input, the decoder hidden states at positions $1, 2, \dots, i-1$ remain unchanged. By leveraging this characteristic of the Transformer decoder, we can compute the output of the decoder *incrementally*.

The feedforward layer, dropout, layer normalization, and residual connection are all elementwise operations, which do not pose challenges in the incremental decoding mode. The only components we need to handle carefully are the two types of attention in a Transformer decoder layer:

- **Self-attention**:  The decoder's self-attention incorporates *causal masking*, ensuring that the hidden state at position $i$ never depends on the hidden state at position $j$ if $j>i$. Suppose we want to compute $\mathrm{CausalSelfAttention}(h_1,h_2,\dots,h_i)$, and we already know $o_1,o_2,\dots,o_{i-1} = \mathrm{CausalSelfAttention}(h_1,h_2,\dots,h_{i-1})$. To obtain $o_i$, we only need to compute the multi-head attention with $[h_i]$ as the query and $[h_1,h_2,\dots,h_i]$ as the key and value.

- **Cross-attention**: In the decoder's cross-attention, the output at each position $o_i$ depends solely on the input at this position $h_i$ and the encoder outputs $\mathrm{enc}(\mathbf{x})$. When a new input vector $h_i$ is appended to the input, we only need to compute the multi-head attention with $[h_i]$ as the query and the encoder output $\mathrm{enc}(\mathbf{x})$ as the key and value to obtain $o_i$.

By adopting this approach, the time complexity of self-attention and cross-attention in each iteration is significantly reduced. Self-attention complexity decreases from $\Theta(n_\text{tgt}^2)$ to $\Theta(n_\text{tgt})$, and cross-attention complexity goes from $\Theta(n_\text{src}n_\text{tgt})$ to $\Theta(n_\text{src})$. As a result, the total time complexity of generation with $\Theta(n_\text{tgt})$ iterations becomes $\Theta(n_\text{src}^2+n_\text{src}n_\text{tgt} + n_\text{tgt}^2)$, which simplifies to $\Theta(n_\text{src}^2 + n_\text{tgt}^2)$.


### Transformer Decoder Layer

Now, let's implement this efficient decoding algorithm step by step. First, we need to modify our implementation of the `TransformerLayer`. We'll add two additional parameters to the `forward` method: `use_cache` and `past_kv`. **Please modify the following implementation of `TransformerLayer` based on these requirements:**

- `use_cache` is a flag indicating whether the efficient generation features should be activated or not. It is set to `True` only when the layer is a decoder layer and the model is in inference mode. If `use_cache` is *not* `True`, the behavior of the method should remain *the same as in Part I*.

- When `use_cache` is true, the efficient generation mode is active. In this case (assume we are generating the $i$-th token):
  - `x` has the shape `[bsz, 1, d_model]`, as we are only feeding one vector $h_i$ (instead of $[h_1,h_2,\dots,h_i]$) into the layer for each sequence.
  - `past_kv` stores the cache of past input states $[h_1,h_2,\dots,h_{i-1}]$. `past_kv` is either `None` (when generating the first token) or has the shape `[bsz, i - 1, d_model]`.
  - The method should return the output state at the $i$-th position, which is another tensor with a shape of `[bsz, 1, d_model]`.
  - The method should also return the new cache `kv`, a tensor with a shape of `[bsz, i, d_model]`, representing $[h_1,h_2,\dots,h_i]$.

*Hint: Consider whether we still need key padding mask and/or causal mask for each multi-head attention.*

*Hint: One part of the implementation may not require any changes.*

In [ ]:
class TransformerLayer(nn.Module):
    def __init__(self, is_decoder, d_model, n_heads, d_ffn, p_drop):
        super().__init__()
        self.is_decoder = is_decoder
        self.self_attn = MultiheadAttention(d_model, n_heads)
        self.self_attn_drop = nn.Dropout(p_drop)
        self.self_attn_ln = nn.LayerNorm(d_model)
        if is_decoder:
            self.cross_attn = MultiheadAttention(d_model, n_heads)
            self.cross_attn_drop = nn.Dropout(p_drop)
            self.cross_attn_ln = nn.LayerNorm(d_model)
        self.fc1 = nn.Linear(d_model, d_ffn)
        self.fc2 = nn.Linear(d_ffn, d_model)
        self.ffn_drop = nn.Dropout(p_drop)
        self.ffn_ln = nn.LayerNorm(d_model)
    
    def forward(
        self,
        x,
        padding_mask,
        encoder_out=None,
        encoder_padding_mask=None,
        use_cache=None,
        past_kv=None
    ):
        residual = x
        ########################################################################
        # TODO: Modify this part of code
        ########################################################################
        # x = self.self_attn(x, x, x, padding_mask, causal=self.is_decoder)
        if past_kv is not None:
            assert self.is_decoder
            kv = torch.cat([past_kv, x], dim=1)
            x = self.self_attn(x, kv, kv, None, causal=False)
        else:
            kv = x
            x = self.self_attn(x, kv, kv, padding_mask, causal=self.is_decoder)
        ########################################################################
        x = self.self_attn_drop(x)
        x = self.self_attn_ln(x + residual)

        if self.is_decoder:
            residual = x
            ####################################################################
            # TODO: Modify this part of code
            ####################################################################
            x = self.cross_attn(x, encoder_out, encoder_out, encoder_padding_mask)
            ####################################################################
            x = self.cross_attn_drop(x)
            x = self.cross_attn_ln(x + residual)

        residual = x
        x = self.fc2(F.relu(self.fc1(x)))
        x = self.ffn_drop(x)
        x = self.ffn_ln(x + residual)
        if use_cache:
            return x, kv
        else:
            return x

The following tests include those from Part I to ensure that your modifications do not alter the module's behavior during training. Additionally, the tests compare your code in token-by-token mode and teacher-forcing mode, verifying that the outputs are consistent in both cases.

In [ ]:
def test_transformer_layer_with_cache():
    torch.manual_seed(750)
    enc_layer0 = nn.TransformerEncoderLayer(128, 4, dim_feedforward=512, dropout=0.0, batch_first=True)
    nn.init.normal_(enc_layer0.self_attn.in_proj_weight, mean=0.0, std=0.05)
    nn.init.normal_(enc_layer0.self_attn.in_proj_bias, mean=0.0, std=0.05)
    nn.init.normal_(enc_layer0.self_attn.out_proj.weight, mean=0.0, std=0.05)
    nn.init.normal_(enc_layer0.self_attn.out_proj.bias, mean=0.0, std=0.05)
    nn.init.normal_(enc_layer0.linear1.weight, mean=0.0, std=0.05)
    nn.init.normal_(enc_layer0.linear1.bias, mean=0.0, std=0.05)
    nn.init.normal_(enc_layer0.linear2.weight, mean=0.0, std=0.05)
    nn.init.normal_(enc_layer0.linear2.bias, mean=0.0, std=0.05)
    enc_layer1 = TransformerLayer(False, 128, 4, 512, 0.0)
    enc_layer1.self_attn.q_proj.weight.data.copy_(enc_layer0.self_attn.in_proj_weight.data[:128, :])
    enc_layer1.self_attn.q_proj.bias.data.copy_(enc_layer0.self_attn.in_proj_bias.data[:128])
    enc_layer1.self_attn.k_proj.weight.data.copy_(enc_layer0.self_attn.in_proj_weight.data[128:256, :])
    enc_layer1.self_attn.k_proj.bias.data.copy_(enc_layer0.self_attn.in_proj_bias.data[128:256])
    enc_layer1.self_attn.v_proj.weight.data.copy_(enc_layer0.self_attn.in_proj_weight.data[256:, :])
    enc_layer1.self_attn.v_proj.bias.data.copy_(enc_layer0.self_attn.in_proj_bias.data[256:])
    enc_layer1.self_attn.o_proj.weight.data.copy_(enc_layer0.self_attn.out_proj.weight.data)
    enc_layer1.self_attn.o_proj.bias.data.copy_(enc_layer0.self_attn.out_proj.bias.data)
    enc_layer1.fc1.weight.data.copy_(enc_layer0.linear1.weight.data)
    enc_layer1.fc1.bias.data.copy_(enc_layer0.linear1.bias.data)
    enc_layer1.fc2.weight.data.copy_(enc_layer0.linear2.weight.data)
    enc_layer1.fc2.bias.data.copy_(enc_layer0.linear2.bias.data)

    torch.manual_seed(800)
    x = torch.randn(4, 5, 128).float()
    x_mask = torch.tensor([[0, 0, 0, 0, 0], [0, 0, 0, 1, 1], [0, 0, 0, 0, 1], [0, 0, 1, 1, 1]]).byte()
    y10 = enc_layer0(x, src_key_padding_mask=x_mask.to(torch.bool)).contiguous()
    y11 = enc_layer1(x, x_mask).contiguous()
    assert vector_all_close(y10[0], y11[0])
    assert vector_all_close(y10[1, :3], y11[1, :3])
    assert vector_all_close(y10[2, :4], y11[2, :4])
    assert vector_all_close(y10[3, :2], y11[3, :2])

    torch.manual_seed(900)
    x = torch.randn(3, 4, 128).float()
    x_mask = torch.tensor([[0, 0, 0, 1], [0, 0, 0, 1], [0, 0, 1, 1]]).byte()
    y20 = enc_layer0(x, src_key_padding_mask=x_mask.to(torch.bool)).contiguous()
    y21 = enc_layer1(x, x_mask).contiguous()
    TO_SAVE["transformer_layer.2.0.shape"] = list(y20.shape)
    TO_SAVE["transformer_layer.2.0.value"] = y20.view(-1)[126: 131].tolist()
    TO_SAVE["transformer_layer.2.1.shape"] = list(y21.shape)
    TO_SAVE["transformer_layer.2.1.value"] = y21.view(-1)[126: 131].tolist()

    torch.manual_seed(950)
    dec_layer0 = nn.TransformerDecoderLayer(128, 4, dim_feedforward=512, dropout=0.0, batch_first=True)
    nn.init.normal_(dec_layer0.self_attn.in_proj_weight, mean=0.0, std=0.05)
    nn.init.normal_(dec_layer0.self_attn.in_proj_bias, mean=0.0, std=0.05)
    nn.init.normal_(dec_layer0.self_attn.out_proj.weight, mean=0.0, std=0.05)
    nn.init.normal_(dec_layer0.self_attn.out_proj.bias, mean=0.0, std=0.05)
    nn.init.normal_(dec_layer0.multihead_attn.in_proj_weight, mean=0.0, std=0.05)
    nn.init.normal_(dec_layer0.multihead_attn.in_proj_bias, mean=0.0, std=0.05)
    nn.init.normal_(dec_layer0.multihead_attn.out_proj.weight, mean=0.0, std=0.05)
    nn.init.normal_(dec_layer0.multihead_attn.out_proj.bias, mean=0.0, std=0.05)
    nn.init.normal_(dec_layer0.linear1.weight, mean=0.0, std=0.05)
    nn.init.normal_(dec_layer0.linear1.bias, mean=0.0, std=0.05)
    nn.init.normal_(dec_layer0.linear2.weight, mean=0.0, std=0.05)
    nn.init.normal_(dec_layer0.linear2.bias, mean=0.0, std=0.05)
    dec_layer1 = TransformerLayer(True, 128, 4, 512, 0.0)
    dec_layer1.self_attn.q_proj.weight.data.copy_(dec_layer0.self_attn.in_proj_weight.data[:128, :])
    dec_layer1.self_attn.q_proj.bias.data.copy_(dec_layer0.self_attn.in_proj_bias.data[:128])
    dec_layer1.self_attn.k_proj.weight.data.copy_(dec_layer0.self_attn.in_proj_weight.data[128:256, :])
    dec_layer1.self_attn.k_proj.bias.data.copy_(dec_layer0.self_attn.in_proj_bias.data[128:256])
    dec_layer1.self_attn.v_proj.weight.data.copy_(dec_layer0.self_attn.in_proj_weight.data[256:, :])
    dec_layer1.self_attn.v_proj.bias.data.copy_(dec_layer0.self_attn.in_proj_bias.data[256:])
    dec_layer1.self_attn.o_proj.weight.data.copy_(dec_layer0.self_attn.out_proj.weight.data)
    dec_layer1.self_attn.o_proj.bias.data.copy_(dec_layer0.self_attn.out_proj.bias.data)
    dec_layer1.cross_attn.q_proj.weight.data.copy_(dec_layer0.multihead_attn.in_proj_weight.data[:128, :])
    dec_layer1.cross_attn.q_proj.bias.data.copy_(dec_layer0.multihead_attn.in_proj_bias.data[:128])
    dec_layer1.cross_attn.k_proj.weight.data.copy_(dec_layer0.multihead_attn.in_proj_weight.data[128:256, :])
    dec_layer1.cross_attn.k_proj.bias.data.copy_(dec_layer0.multihead_attn.in_proj_bias.data[128:256])
    dec_layer1.cross_attn.v_proj.weight.data.copy_(dec_layer0.multihead_attn.in_proj_weight.data[256:, :])
    dec_layer1.cross_attn.v_proj.bias.data.copy_(dec_layer0.multihead_attn.in_proj_bias.data[256:])
    dec_layer1.cross_attn.o_proj.weight.data.copy_(dec_layer0.multihead_attn.out_proj.weight.data)
    dec_layer1.cross_attn.o_proj.bias.data.copy_(dec_layer0.multihead_attn.out_proj.bias.data)
    dec_layer1.fc1.weight.data.copy_(dec_layer0.linear1.weight.data)
    dec_layer1.fc1.bias.data.copy_(dec_layer0.linear1.bias.data)
    dec_layer1.fc2.weight.data.copy_(dec_layer0.linear2.weight.data)
    dec_layer1.fc2.bias.data.copy_(dec_layer0.linear2.bias.data)

    torch.manual_seed(1000)
    x = torch.randn(4, 5, 128).float()
    e = torch.randn(4, 3, 128).float()
    x_mask = torch.tensor([[0, 0, 0, 0, 0], [0, 0, 0, 1, 1], [0, 0, 0, 0, 1], [0, 0, 1, 1, 1]]).byte()
    e_mask = torch.tensor([[0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 1]]).byte()
    y30 = dec_layer0(x, e, tgt_mask=future_mask[:5, :5], tgt_key_padding_mask=x_mask.to(torch.bool), memory_key_padding_mask=e_mask.to(torch.bool)).contiguous()
    y31 = dec_layer1(x, x_mask, e, e_mask).contiguous()
    assert vector_all_close(y30[0], y31[0])
    assert vector_all_close(y30[1, :3], y31[1, :3])
    assert vector_all_close(y30[2, :4], y31[2, :4])
    assert vector_all_close(y30[3, :2], y31[3, :2])

    torch.manual_seed(1100)
    x = torch.randn(3, 4, 128).float()
    e = torch.randn(3, 3, 128).float()
    x_mask = torch.tensor([[0, 0, 0, 0], [0, 0, 0, 1], [0, 0, 1, 1]]).byte()
    e_mask = torch.tensor([[0, 0, 0], [0, 0, 0], [0, 0, 1]]).byte()
    y40 = dec_layer0(x, e, tgt_mask=future_mask[:4, :4], tgt_key_padding_mask=x_mask.to(torch.bool), memory_key_padding_mask=e_mask.to(torch.bool)).contiguous()
    y41 = dec_layer1(x, x_mask, e, e_mask).contiguous()
    TO_SAVE["transformer_layer.4.0.shape"] = list(y40.shape)
    TO_SAVE["transformer_layer.4.0.value"] = y40.view(-1)[126: 131].tolist()
    TO_SAVE["transformer_layer.4.1.shape"] = list(y41.shape)
    TO_SAVE["transformer_layer.4.1.value"] = y41.view(-1)[126: 131].tolist()

    torch.manual_seed(1200)
    x = torch.randn(5, 6, 128).float()
    e = torch.randn(5, 3, 128).float()
    x_len = [4, 5, 3, 6, 2]
    x_mask = torch.tensor([[0, 0, 0, 0, 1, 1], [0, 0, 0, 0, 0, 1], [0, 0, 0, 1, 1, 1], [0, 0, 0, 0, 0, 0], [0, 0, 1, 1, 1, 1]]).byte()
    e_mask = torch.tensor([[0, 0, 1], [0, 1, 1], [0, 0, 1], [0, 0, 0], [0, 0, 0]]).byte()
    y50 = dec_layer0(x, e, tgt_mask=future_mask[:6, :6], tgt_key_padding_mask=x_mask.to(torch.bool), memory_key_padding_mask=e_mask.to(torch.bool)).contiguous()
    y51 = []
    for i in range(x.size(1)):
        y51_cur, cur_kv = dec_layer1(x[:, i: i + 1, :], None, e, e_mask, use_cache=True, past_kv=x[:, :i, :] if i > 0 else None)
        y51.append(y51_cur)
    y51 = torch.cat(y51, dim=1)
    for i, l in enumerate(x_len):
        assert vector_all_close(y51[i, :l], y50[i, :l])
        assert vector_all_close(cur_kv[i, :l], x[i, :l])
    
    torch.manual_seed(1300)
    x = torch.randn(4, 5, 128).float()
    e = torch.randn(4, 3, 128).float()
    x_len = [4, 5, 3, 2]
    x_mask = torch.tensor([[0, 0, 0, 0, 1], [0, 0, 0, 0, 0], [0, 0, 0, 1, 1], [0, 0, 1, 1, 1]]).byte()
    e_mask = torch.tensor([[0, 0, 1], [0, 1, 1], [0, 0, 1], [0, 0, 0]]).byte()
    y60 = dec_layer0(x, e, tgt_mask=future_mask[:5, :5], tgt_key_padding_mask=x_mask.to(torch.bool), memory_key_padding_mask=e_mask.to(torch.bool)).contiguous()
    y61 = []
    for i in range(x.size(1)):
        y61_cur, cur_kv = dec_layer1(x[:, i: i + 1, :], None, e, e_mask, use_cache=True, past_kv=x[:, :i, :] if i > 0 else None)
        y61.append(y61_cur)
    y61 = torch.cat(y61, dim=1).contiguous()
    TO_SAVE["transformer_layer.6.0.shape"] = list(y60.shape)
    TO_SAVE["transformer_layer.6.0.value"] = y60.view(-1)[380: 385].tolist()
    TO_SAVE["transformer_layer.6.1.shape"] = list(y61.shape)
    TO_SAVE["transformer_layer.6.1.value"] = y61.view(-1)[380: 385].tolist()

test_transformer_layer_with_cache()

### Transformer Stack

In Part I, we integrated Transformer layers directly into the `Transformer` module. However, to maintain readability as we add more code to the `Transformer` class in this assignment, we will introduce an additional layer of encapsulation. Specifically, we will implement the `TransformerStack`. A Transformer stack can function as either a Transformer encoder or a Transformer decoder, depending on the value of `is_decoder`.

We provide an implementation of `TransformerStack` based on the solution to Part I. Now, we need to **modify its `forward` method to support efficient generation with a cache**. Two new parameters are added to the method: `use_cache` and `past_key_values`.

- `use_cache` is a flag that determines whether to enable efficient generation or not.

- If `use_cache` is on, `past_key_values` is a list with a length equal to the number of (decoder) layers in the stack. Specifically, `past_key_values[i]` stores the cache (`past_kv` for `TransformerLayer.forward`) for layer `i`.

- When `use_cache` is on, `input_ids` will contain only the input ids for the current generation step, resulting in a shape of `[bsz, 1]`.

*Hint: Consider modifying the positional embeddings added to the token embeddings. Think about what would happen to positional embeddings if `input_ids.shape == [bsz, 1]` in every step without any changes.*

In [ ]:
class TransformerStack(PreTrainedModel):
    def __init__(self, config, emb_word, emb_pos, emb_ln, is_decoder):
        super().__init__(config)
        self.emb_word = emb_word
        self.emb_pos = emb_pos
        self.emb_ln = emb_ln
        self.is_decoder = is_decoder
        self.layers = nn.ModuleList([
            TransformerLayer(
                is_decoder,
                self.config.d_model,
                self.config.d_model // self.config.d_kv,
                self.config.d_ff,
                self.config.dropout_rate
            )
            for _ in range(
                self.config.num_decoder_layers
                if is_decoder
                else self.config.num_encoder_layers
            )
        ])
    
    def make_positions(self, input_ids, padding_mask):
        return einops.repeat(
            torch.arange(padding_mask.size(1)).to(input_ids),
            "l -> b l",
            b=padding_mask.size(0)
        ).contiguous()
    
    def forward(
        self,
        input_ids,
        encoder_hidden_states=None,
        encoder_padding_mask=None,
        use_cache=None,
        past_key_values=None,
        **kwargs
    ):
        if past_key_values is None:
            past_key_values = [None for _ in range(len(self.layers))]
        padding_mask = input_ids.eq(self.config.pad_token_id).byte()
        pos = self.make_positions(input_ids, padding_mask)
        ########################################################################
        # TODO: modify this part of code
        ########################################################################
        # state = self.emb_ln(self.emb_word(input_ids) + self.emb_pos(pos))
        if use_cache and past_key_values[0] is not None:
            pos += past_key_values[0].size(1)
        state = self.emb_ln(self.emb_word(input_ids) + self.emb_pos(pos))
        ########################################################################
        for i, layer in enumerate(self.layers):
            ####################################################################
            # TODO: modify this part of code
            ####################################################################
            # state = layer(
            #     state,
            #     padding_mask,
            #     encoder_hidden_states,
            #     encoder_padding_mask
            # )
            state = layer(
                state,
                padding_mask,
                encoder_hidden_states,
                encoder_padding_mask,
                use_cache,
                past_key_values[i]
            )
            if use_cache:
                state, past_key_values[i] = state
            ####################################################################
        if use_cache:
            return state, padding_mask, past_key_values
        else:
            return state, padding_mask, None

The following tests compare your implementation of Transformer stacks in token-by-token mode and teacher-forcing mode, verifying that the outputs are consistent in both cases.

In [ ]:
def test_transformer_stack_with_cache():
    config = PretrainedConfig(
        is_encoder_decoder=True,
        use_cache=True,
        decoder_start_token_id=tokenizer.eos_token_id,
        vocab_size=tokenizer.vocab_size,
        pad_token_id=tokenizer.pad_token_id,
        eos_token_id=tokenizer.eos_token_id,
        n_positions=1024,
        num_decoder_layers=2,
        d_model=128,
        d_ff=512,
        d_kv=32,
        dropout_rate=0.0
    )

    torch.manual_seed(1400)
    emb_word = nn.Embedding(config.vocab_size, config.d_model)
    emb_pos = nn.Embedding(config.n_positions, config.d_model)
    emb_word.weight.data.uniform_(-0.05, 0.05)
    emb_pos.weight.data.uniform_(-0.05, 0.05)
    emb_ln = nn.LayerNorm(config.d_model)

    stack = TransformerStack(config, emb_word, emb_pos, emb_ln, True)

    input_ids = torch.randint(10, 100, (5, 6))
    padding_mask = torch.tensor([[0, 0, 0, 0, 1, 1], [0, 0, 0, 0, 0, 1], [0, 0, 0, 1, 1, 1], [0, 0, 0, 0, 0, 0], [0, 0, 1, 1, 1, 1]]).byte()
    input_ids[padding_mask] = config.pad_token_id
    encoder_hidden_states = torch.randn(5, 3, 128)
    encoder_padding_mask = torch.tensor([[0, 0, 1], [0, 1, 1], [0, 0, 1], [0, 0, 0], [0, 0, 0]]).byte()
    x_len = [4, 5, 3, 6, 2]

    state_1, padding_mask_1, past_key_values_1 = stack(input_ids, encoder_hidden_states, encoder_padding_mask)
    assert list(state_1.shape) == [5, 6, 128]
    assert (padding_mask_1 == padding_mask).all().item()
    assert past_key_values_1 is None

    state_2, padding_mask_2, past_key_values_2 = [], [], None
    for i in range(input_ids.size(1)):
        state_, padding_mask_, past_key_values_ = stack(
            input_ids[:, i: i + 1],
            encoder_hidden_states,
            encoder_padding_mask,
            use_cache=True,
            past_key_values=past_key_values_2
        )
        state_2.append(state_)
        padding_mask_2.append(padding_mask_)
        past_key_values_2 = past_key_values_
    state_2 = torch.cat(state_2, dim=1)
    padding_mask_2 = torch.cat(padding_mask_2, dim=1)

    assert list(state_2.shape) == [5, 6, 128]
    for i in range(input_ids.size(0)):
        assert vector_all_close(state_1[i, :x_len[i]], state_2[i, :x_len[i]])
    assert (padding_mask_2 == padding_mask).all().item()
    assert len(past_key_values_2) == 2 and list(past_key_values_2[0].shape) == [5, 6, 128]

    input_ids = torch.randint(10, 100, (3, 4))
    padding_mask = torch.tensor([[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 1]]).byte()
    input_ids[padding_mask] = config.pad_token_id
    encoder_hidden_states = torch.randn(3, 5, 128)
    encoder_padding_mask = torch.tensor([[0, 0, 0, 0, 1], [0, 0, 0, 1, 1], [0, 0, 0, 0, 0]]).byte()
    x_len = [4, 4, 3]

    state_1, padding_mask_1, past_key_values_1 = stack(input_ids, encoder_hidden_states, encoder_padding_mask)
    TO_SAVE["transformer_layer.2.state_1.shape"] = list(state_1.shape)
    TO_SAVE["transformer_layer.2.state_1.value"] = state_1.view(-1)[380: 385].tolist()

    state_2, padding_mask_2, past_key_values_2 = [], [], None
    for i in range(input_ids.size(1)):
        state_, padding_mask_, past_key_values_ = stack(
            input_ids[:, i: i + 1],
            encoder_hidden_states,
            encoder_padding_mask,
            use_cache=True,
            past_key_values=past_key_values_2
        )
        state_2.append(state_)
        padding_mask_2.append(padding_mask_)
        past_key_values_2 = past_key_values_
    state_2 = torch.cat(state_2, dim=1)
    padding_mask_2 = torch.cat(padding_mask_2, dim=1)

    TO_SAVE["transformer_layer.2.state_2.shape"] = list(state_2.shape)
    TO_SAVE["transformer_layer.2.state_2.value"] = state_2.view(-1)[380: 385].tolist()

test_transformer_stack_with_cache()

### Transformer

Now, let's bring everything together. The following code cell provides the definition of the Transformer model. The `Transformer` class consists of the embedding layers and Transformer stacks you defined earlier. Additionally, we have implemented the interfaces for you to prepare inputs for fast generation and reorder caches according to the API of the `trainer` in the `transformers` library.


In [ ]:
class Transformer(PreTrainedModel):
    def __init__(self, config):
        super().__init__(config)
        self.emb_word = nn.Embedding(self.config.vocab_size, self.config.d_model)
        self.emb_pos = nn.Embedding(self.config.n_positions, self.config.d_model)
        self.emb_word.weight.data.uniform_(-0.05, 0.05)
        self.emb_pos.weight.data.uniform_(-0.05, 0.05)
        self.emb_ln = nn.LayerNorm(self.config.d_model)
        self.encoder = TransformerStack(config, self.emb_word, self.emb_pos, self.emb_ln, False)
        self.decoder = TransformerStack(config, self.emb_word, self.emb_pos,self. emb_ln, True)
        self.lm_head = nn.Linear(self.config.d_model, self.config.vocab_size)
        self.lm_head.weight = self.emb_word.weight
        self.criterion = nn.CrossEntropyLoss(ignore_index=-100)
    
    def forward(
        self,
        input_ids=None,
        decoder_input_ids=None,
        encoder_outputs=None,
        labels=None,
        use_cache=None,
        past_key_values=None,
        **kwargs
    ):
        assert not (set(kwargs.keys()) - {"output_attentions", "output_hidden_states", "attention_mask", "return_dict"}), kwargs
        if encoder_outputs is None:
            encoder_outputs = self.encoder(input_ids)
        enc_state, enc_padding_mask, _ = encoder_outputs
        if decoder_input_ids is None and labels is not None:
            decoder_input_ids = labels.new_zeros(labels.shape)
            decoder_input_ids[:, 1:] = labels[:, :-1].clone()
            decoder_input_ids[:, 0] = self.config.eos_token_id
            decoder_input_ids.masked_fill_(decoder_input_ids == -100, self.config.pad_token_id)
        dec_state, dec_padding_mask, dec_present_kv = self.decoder(
            decoder_input_ids,
            enc_state,
            enc_padding_mask,
            use_cache,
            past_key_values if use_cache else None
        )
        lm_logits = self.lm_head(dec_state)
        loss = None
        if labels is not None:
            loss = self.criterion(lm_logits.view(-1, lm_logits.size(-1)), labels.view(-1))
        return Seq2SeqLMOutput(
            loss=loss,
            logits=lm_logits,
            past_key_values=dec_present_kv
        )
    
    def get_encoder(self):
        return self.encoder

    def get_decoder(self):
        return self.decoder
    
    def prepare_inputs_for_generation(self, input_ids, encoder_outputs=None, use_cache=None, past=None, **kwargs):
        # print("prepare_inputs_for_generation", kwargs)
        if past is not None:
            input_ids = input_ids[:, -1:]

        return {
            "decoder_input_ids": input_ids,
            "encoder_outputs": encoder_outputs,
            "use_cache": use_cache,
            "past_key_values": past
        }
    
    @staticmethod
    def _expand_inputs_for_generation(input_ids, expand_size=1, encoder_outputs=None, **kwargs):
        assert (
            set(kwargs.keys()) == {"is_encoder_decoder", "use_cache", "output_hidden_states", "output_attentions"}
            or set(kwargs.keys()) == {"is_encoder_decoder", "use_cache", "output_hidden_states", "output_attentions", "attention_mask"}
        ), set(kwargs.keys())
        input_ids = einops.repeat(input_ids, "b l -> (b beam) l", beam=expand_size)
        enc_state, enc_padding_mask, _ = encoder_outputs
        enc_state = einops.repeat(enc_state, "b l d -> (b beam) l d", beam=expand_size)
        enc_padding_mask = einops.repeat(enc_padding_mask, "b l -> (b beam) l", beam=expand_size)
        kwargs["encoder_outputs"] = enc_state, enc_padding_mask, None
        return input_ids, kwargs

    def _reorder_cache(self, past, beam_idx):
        return [
            past_kv.index_select(0, beam_idx.to(past_kv.device))
            for past_kv in past
        ]

Here, we define and load the Transformer model, which comprises 6 encoder layers and 6 decoder layers. The model has been trained on the summarization dataset for 5 epochs. The expected output of this code cell is `<All keys matched successfully>`.

In [ ]:
config = PretrainedConfig(
    is_encoder_decoder=True,
    use_cache=True,
    decoder_start_token_id=tokenizer.eos_token_id,
    vocab_size=tokenizer.vocab_size,
    pad_token_id=tokenizer.pad_token_id,
    eos_token_id=tokenizer.eos_token_id,
    n_positions=1024,
    num_encoder_layers=6,
    num_decoder_layers=6,
    d_model=512,
    d_ff=2048,
    d_kv=64,
    dropout_rate=0.1
)

model = Transformer(config)
model.load_state_dict(torch.load("/content/drive/MyDrive/cs182_hw_summarization_part2/hw11.pytorch_model.bin", map_location="cpu"))

### Evaluation Metric: ROUGE Score

ROUGE stands for Recall-Oriented Understudy for Gisting Evaluation. It is a widely-used metric for assessing the quality of generated summaries by comparing them to reference summaries. The ROUGE score consists of several variants, including ROUGE-N (measuring n-gram overlap) and ROUGE-L (measuring the longest common subsequence). These variants help quantify the similarity between the generated summary and the reference summary, with higher scores indicating better summarization performance.

In [ ]:
rouge = evaluate.load("rouge")

def compute_metrics(tokenizer, eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    result = rouge.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)

    return {k: round(v, 4) for k, v in result.items()}

compute_metrics_fn = functools.partial(compute_metrics, tokenizer)

Let's evaluate the model you just loaded on the validation dataset. If your implementation of the Transformer is correct, you should expect a ROUGE-1 score of approximately `18.81` (remember to multiply the output of the `transformers` library by 100 to obtain the correct ROUGE score).

In [ ]:
training_args = Seq2SeqTrainingArguments(
    output_dir="my_xsum_model",
    learning_rate=1e-4,
    weight_decay=0.01,
    warmup_ratio=0.1,

    per_device_train_batch_size=16,
    gradient_accumulation_steps=2,
    dataloader_drop_last=True,
    per_device_eval_batch_size=16,
    num_train_epochs=5,

    predict_with_generate=True,
    generation_num_beams=4,
    push_to_hub=False,
    logging_strategy="steps",
    logging_first_step=True,
    logging_steps=1,
    save_strategy="epoch",
    evaluation_strategy="epoch",
    fp16=True,
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=dataset_train,
    eval_dataset=dataset_validation,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics_fn
)

trainer.evaluate()

#### Question

In your submission for the written assignment, **please report the ROUGE-1, ROUGE-2, and ROUGE-L scores of this model**.

## Finetuning

In modern NLP, rather than training a Transformer model from scratch for a specific task, it is common to *finetune* a pretrained language model, such as BERT (encoder), GPT (decoder), or T5 (encoder-decoder). Finetuning typically reduces training time, improves data efficiency, and consequently leads to better performance.

Next, we will finetune a T5 model for the summarization task. We will initialize the Transformer encoder-decoder model with the parameters of a pretrained *T5-Small* model, which is also a 6-layer encoder and 6-layer decoder Transformer. Unlike transfer learning for image classification or text classification, there is no need to initialize a new classification head for the sequence-to-sequence task, as the language-modeling head can be reused for decoding output sequences. This makes finetuning for sequence-to-sequence tasks with language models remarkably straightforward.

In [ ]:
model = T5ForConditionalGeneration.from_pretrained("t5-small")

### Learning Rate in Finetuning

In finetuning, we typically use a learning rate lower than the one used when training the model initially. To find the learning rate that T5 uses for pretraining and finetuning, refer to the T5 paper (https://arxiv.org/pdf/1910.10683.pdf). Keep in mind that the paper is 67 pages long, so you may not want to read it in its entirety before answering this question. Feel free to skim through most sections and make use of the table of contents and keyword lookups in your PDF reader.

#### Question

**Which optimizer is used to train T5? What is the peak learning rate that T5 uses for pretraining and finetuning, respectively?** Include your answer in your submission of the written assignment.

### Running Finetuning

Let's finetune our model using the Adam optimizer and a linear learning rate schedule with 10% warmup steps. We set the learning rate to `1e-4` and finetune for 3 epochs. Note that the previous model you evaluated is trained from scratch for 5 epochs.

In [ ]:
training_args = Seq2SeqTrainingArguments(
    output_dir="t5_small_for_xsum_2",
    learning_rate=1e-4,
    weight_decay=0.01,
    warmup_ratio=0.1,

    per_device_train_batch_size=16,
    gradient_accumulation_steps=2,
    dataloader_drop_last=True,
    per_device_eval_batch_size=16,
    num_train_epochs=3,

    predict_with_generate=True,
    generation_num_beams=4,
    push_to_hub=False,
    logging_dir="t5_small_for_xsum_2/logs",
    logging_strategy="steps",
    logging_first_step=True,
    logging_steps=1,
    save_strategy="epoch",
    evaluation_strategy="epoch",
    fp16=True,
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=dataset_train,
    eval_dataset=dataset_validation,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics_fn
)

trainer.train()

#### Question

In your submission for the written assignment, **please report the ROUGE-1, ROUGE-2, and ROUGE-L scores of your finetuned model**. **Compare it with the model you loaded that was trained from scratch.**

### Case Analysis

Let's examine some examples of the model's summarization outputs. We'll feed a few examples from the test dataset into the model and assess whether it produces reasonable outputs.

In [ ]:
idx = 10
print("Document:")
print(dataset_test[idx]['document'])
print()
print("Reference summary:")
print(dataset_test[idx]['summary'])
print()

In [ ]:
print("Your model's summary:")
print(tokenizer.decode(
    model.generate(
        torch.tensor([dataset_test[idx]['input_ids']]).to(model.device),
        num_beams=8
    )[0],
    skip_special_tokens=True,
    clean_up_tokenization_spaces=True
))
print()

In [ ]:
idx = 30
print("Document:")
print(dataset_test[idx]['document'])
print()
print("Reference summary:")
print(dataset_test[idx]['summary'])
print()

In [ ]:
print("Your model's summary:")
print(tokenizer.decode(
    model.generate(
        torch.tensor([dataset_test[idx]['input_ids']]).to(model.device),
        num_beams=8
    )[0],
    skip_special_tokens=True,
    clean_up_tokenization_spaces=True
))
print()

#### Question

**Feed each of these sentences with appropriate prompts into ChatGPT or other instruction-finetuned large language models.** Include **your input** for the large language model, **the specific large language model** you use, and **its outputs** in your submission of the written assignment. **Qualitatively compare the generated outputs of your model with the large language model.** What do you think is the main reason for the difference?

## Submission

You are done with this coding assignment. Please download `submission_log.json` and submit it to Gradescope.

In [ ]:
with open("submission_logs.json", "w", encoding="utf-8") as f:
    json.dump(TO_SAVE, f)